In [1]:
#push!(LOAD_PATH,"julia/src")
#import Simulation

4-element Array{String,1}:
 "@"
 "@v#.#"
 "@stdlib"
 "julia/src"

In [56]:
include("julia/src/Simulation/Simulation.jl") # might need some time to compile

Main.Simulation

In [57]:
using Random
#loads and pre-processes data that is constant during the simulation time
params = Simulation.load_params(
    MersenneTwister(3),
    population_path="data/simulations/wroclaw-population-orig.csv.gz", 
    incubation_time_samples_path="test/models/assets/incubation_period_distribution.npy", 
    t0_to_t1_samples_path="test/models/assets/t1_distribution.npy",
    t0_to_t2_samples_path="test/models/assets/t1_t2_distribution.npy");

params.progressions[1] |> display
params.household_ptrs[1] |> display

Main.Simulation.Progression(Main.Simulation.Severe, 9.0f0, 13.25f0, 16.0f0, 30.0f0, NaN32)

(0x00000001, 0x00000001)

In [58]:
state = Simulation.SimState(params.progressions |> length, seed=123)

push!(state.queue, Simulation.Event(Val(Simulation.OutsideInfectionEvent),0.0,1))


@time for iter  in 0:100
    #println("iteration=$iter")
    if isempty(state.queue)
        @info "Empty queue after $iter events"
        break
    end
    
    event = pop!(state.queue)
    state.time = Simulation.time(event)
    #println(event.event_kind)   
    result = Simulation.execute!(state, params, event)
    
end
event = pop!(state.queue)
state.time = Simulation.time(event)
println(event.event_kind)
@code_warntype Simulation.execute!(state, params, event)

#event |> display

#id = 0x000781a0
#state.individuals[id]
#Simulation.householdof(params, id) |> collect

  0.760575 seconds (903.28 k allocations: 45.478 MiB)
TransmissionEvent
Variables
  #self#::Core.Compiler.Const(Main.Simulation.execute!, false)
  state::Main.Simulation.SimState
  params::Main.Simulation.SimParams
  event::Main.Simulation.Event
  kind::Main.Simulation.EventKind

Body::Bool
1 ─ %1 = Main.Simulation.Bool::Core.Compiler.Const(Bool, false)
│   %2 = Base.getproperty(event, :event_kind)::Main.Simulation.EventKind
│   %3 = Base.convert(Main.Simulation.EventKind, %2)::Main.Simulation.EventKind
│        (kind = Core.typeassert(%3, Main.Simulation.EventKind))
│   %5 = Main.Simulation.Val(kind)::Val{_A} where _A
│   %6 = Main.Simulation.execute!(%5, state, params, event)::Any
│   %7 = Core.typeassert(%6, Main.Simulation.Bool)::Bool
│   %8 = Base.convert(%1, %7)::Bool
│   %9 = Core.typeassert(%8, %1)::Bool
└──      return %9


In [13]:
Simulation.Event(Val(Simulation.OutsideInfectionEvent),0.0,1)

Main.Simulation.Event(0.0f0, 0x00000001, 0x00000000, Main.Simulation.OutsideInfectionEvent, Main.Simulation.OutsideContact)

In [59]:
Val(5) |> typeof

Val{5}

In [335]:
id =0x00094a94
params.progressions[id] |> display
state.individuals[id] |> display

Main.Simulation.Progression(Main.Simulation.Mild, 1.0f0, 1.25f0, NaN32, 1.25f0, NaN32)

Main.Simulation.IndividualState(Main.Simulation.MildSymptoms, Main.Simulation.Free, 0, false)

In [39]:
Val(Simulation.DeathEvent) |> isbits

true

In [52]:
## creates mutable simulation state
state = Simulation.SimState(params.progressions |> length, seed=123)

# add initial event (infect the person no1 at time 0)
push!(state.queue, Simulation.OutsideInfectionEvent(0.0,1))
# (the first person must be infectious for significant amount of time for the epidemics to kick off)

# the first run includes compilation time, run twice to get meaningful results (precompilation will be added soon)
@time Simulation.simulate!(state, params)


UndefVarError: UndefVarError: params_id not defined

In [279]:
using DataStructures
i1, i2 = searchequalrange(state.infections, 1)
state.infections[i1]


Main.Simulation.TransmissionEvent(18.69558f0, 491936, 1, Main.Simulation.ConstantKernelContact)

In [10]:
infections = SortedMultiDict{Int32,Int32}()
push!(infections, 0=>1)
push!(infections, 1=>2)
push!(infections, 1=>3)
push!(infections, 2=>4)
push!(infections, 3=>5)
push!(infections, 3=>6)
for (k,v) in inclusive(infections, searchequalrange(infections, 3)...)
    println("$k ---> $v")
end

3 ---> 5
3 ---> 6


In [8]:
inclusive(infections, i1, i2) |> sizeof

24